In [ ]:
import os, glob, json, csv, subprocess, sys, re, operator, itertools
import datetime
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Iterable
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository and analyze directory

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/hbase/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

In [ ]:
bug_links = open(analyze_dir + "document/bug_links.txt", "r")
bugid_list = bug_links.read().split('\n')
print ("Found " + str(len(bugid_list)) + " bug ids'")

# Define function to apply git command

In [ ]:
def execute_command(cmd, work_dir):
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Applying git command

In [ ]:
print (bugid_list)

In [ ]:
result = []
for b, bugid in enumerate(bugid_list):
    sys.stdout.write("\r%i" %(b+1) + " extracting bug id " + bugid)
    sys.stdout.flush()
    git_cmd = "git log --oneline | grep -w '" + bugid + "' > " + analyze_dir + "step_3/future_bugs/logs/" + bugid + "_log.txt" 
    execute_command(git_cmd, repository)

    log = open(analyze_dir + "step_3/future_bugs/logs/" + bugid + "_log.txt")
    bug_log = (log.read().split('\n'))[:-1]
    result.append(bug_log)
result

In [ ]:
multiple_bugs = []
less_bugs = []

for f, bug_item in enumerate(result):
    length = len(bug_item)
    if length <= 2:
        less_bugs.append(bug_item)
    else:
        multiple_bugs.append(bug_item)

In [ ]:
print (len(less_bugs))
print (len(multiple_bugs))

In [ ]:
less_bugs

In [ ]:
multiple_bugs    

In [ ]:
multi_bugs = []
for n, multi in enumerate(multiple_bugs):
    mbg = []
    for o, m_bugs in enumerate(multi):
        com = m_bugs.split(" ")
        log = ' '.join(com[1:])
        logs = [com[0], log]
        mbg.extend(logs)
    multi_bugs.append(mbg)

In [ ]:
multi_bugs[0][0]

In [ ]:
last_com = multi_bugs[0][0]
first_com = multi_bugs[0][len(multi_bugs[0])-4]
print(last_com, first_com)

In [ ]:
bg = re.search("(HBASE-)[0-9]*", multi_bugs[49][len(multi_bugs[49])-1])
bg = bg.group()
bg

In [ ]:
futurebugs_qty = []
for x, xx in enumerate(multi_bugs):
    bug_pattern = re.search("(HBASE-)[0-9]*", xx[len(xx)-1])
    bug_pattern = bug_pattern.group()
    qty = int((len(xx)/2)-1)
    
    for y, yy in enumerate(xx):
        last_com = str(xx[0])
        first_com = str(xx[len(xx)-4])
    
    temp = [bug_pattern, qty, first_com, last_com]
    futurebugs_qty.append(temp)

futurebugs_qty

In [ ]:
for d, dt in enumerate(futurebugs_qty):
    #date_t = []
    for q in range(2,4):
        git_cmd = "git show -s --format=%ci " + dt[q]
        tmp = str(execute_command(git_cmd, repository)).replace("b'",'').replace("(",'').replace(")",'').split("\\n")
        del tmp[len(tmp)-1]
        tmp = tmp[0].split(" ")
        futurebugs_qty[d].insert(len(dt), tmp[0])
        futurebugs_qty[d].insert(len(dt), tmp[1])
        futurebugs_qty[d].insert(len(dt), tmp[2])

In [ ]:
print (futurebugs_qty)

In [ ]:
with open(analyze_dir + "step_3/future_bugs/quantity/futurebugs.csv",'w') as csvfile:
    header = ['bug_id', '#future_bugs', 'first_futurecommitID', 'last_futurecommitID', 'firstdate_CID', 'firsttime_CID',
              'firsttz_CID', 'lastdate_CID', 'lasttime_CID', 'lasttz_CID']
    writers = csv.writer(csvfile)
    writers.writerow(header)
    for l in futurebugs_qty:
        writers.writerow(l)

In [ ]:
df_future = pd.read_csv(analyze_dir + "step_3/future_bugs/quantity/futurebugs.csv")
df_future = df_future[header]
df_future

In [ ]:
last = datetime.datetime.strptime(df_future.iloc[0][7] + " " + df_future.iloc[0][8], "%Y-%m-%d %H:%M:%S")
first = datetime.datetime.strptime(df_future.iloc[0][4] + " " + df_future.iloc[0][5], "%Y-%m-%d %H:%M:%S")
print (last - first)

In [ ]:
with open(analyze_dir + "step_3/future_bugs/quantity/futurebugs.csv",'r') as csvinput:
    with open(analyze_dir + "step_3/future_bugs/quantity/timespan_futurebugs.csv", 'w') as csvoutput:
        writers = csv.writer(csvoutput, lineterminator='\n')
        reader = csv.reader(csvinput)

        all_data = []
        row = next(reader)
        row.append('timespan_betweenbugs')
        all_data.append(row)

        for row in reader:
            last = datetime.datetime.strptime(row[7] + " " + row[8], "%Y-%m-%d %H:%M:%S")
            first = datetime.datetime.strptime(row[4] + " " + row[5], "%Y-%m-%d %H:%M:%S")
            row.append(last - first)
            all_data.append(row)

        writers.writerows(all_data)

In [ ]:
df_timespan = pd.read_csv(analyze_dir + "step_3/future_bugs/quantity/timespan_futurebugs.csv")
columns = ['bug_id', '#future_bugs', 'first_futurecommitID', 'last_futurecommitID', 'firstdate_CID', 'firsttime_CID',
          'firsttz_CID', 'lastdate_CID', 'lasttime_CID', 'lasttz_CID', 'timespan_betweenbugs']
df_timespan = df_timespan[columns]
df_timespan

In [ ]:
str(datetime.timedelta(seconds=666))